In [1]:
#Import Dependencies
import pandas as pd
import numpy as np

In [2]:
# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


# Player Count

In [3]:
# Calculate toal number of players
player_count = purchase_data['SN'].nunique()
player_count_df = pd.DataFrame({"Total Players": [player_count]}, columns= ["Total Players"])
player_count_df

,Total Players
0,576


# Purchasing Analysis (Total)

In [4]:
# Calculate number of unique items, average purchase, total purchases, and total revenue
unique_items = purchase_data['Item ID'].nunique()
average_price = purchase_data['Price'].mean()
total_purchases = purchase_data['Price'].count()
total_revenue = purchase_data['Price'].sum()


# Create a summary data frame to hold the results
purchasing_analysis = {
    'Number of Unique Items': [unique_items],
    'Average Price': [average_price],
    'Number of Purchases' : [total_purchases],
    'Total Revenue': [total_revenue]}

purchasing_analysis_df = pd.DataFrame(purchasing_analysis)

# Rename and format columns
purchasing_analysis_df.style.format({'Average Price' : '${:.2f}', 'Total Revenue' : '${:.2f}'})

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


# Gender Demographics 

In [5]:
# Count male, female, other/non-disclosed players
no_duplicates_gender = purchase_data.drop_duplicates(['SN'])
player_gender_counts = no_duplicates_gender['Gender'].value_counts()

# Calculate percentage of male, female, other/non-disclosed players
player_total = no_duplicates_gender['Gender'].count()
gender_percentages = player_gender_counts / player_total * 100

# Round and rename columns
gender_percentages_final = gender_percentages.round(2)
gender_demographics = pd.DataFrame({"Total Count" : player_gender_counts, "Percentage of Players" : gender_percentages_final})
gender_demographics


,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


# Purchasing Analysis (Gender)

In [6]:
# Calculate purchase count and total purchases
purchase_count_gender = pd.DataFrame(purchase_data.groupby('Gender')['Gender'].count())
total_purchase_gender = pd.DataFrame(purchase_data.groupby('Gender')['Price'].sum())

# Merge dataframes for purchase count and total purchases
purchasing_analysis_gender = pd.merge(purchase_count_gender, total_purchase_gender, left_index = True, right_index = True)

# Rename columns
purchasing_analysis_gender.columns = ['Purchase Count' , 'Total Purchase Value']

# Calculate average purchase price and average total purchase per person and add these new columns
purchasing_analysis_gender['Average Purchase Price'] = purchasing_analysis_gender['Total Purchase Value'] / purchasing_analysis_gender['Purchase Count']
purchasing_analysis_gender['Avg Total Purchase per Person'] = purchasing_analysis_gender['Total Purchase Value'] / player_gender_counts

# Format columns 
purchasing_analysis_gender.style.format({'Total Purchase Value' : '${:,.2f}', 'Average Purchase Price' : '${:.2f}', 'Avg Total Purchase per Person' : '${:.2f}'})


,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Gender,,,,
Female,113,$361.94,$3.20,$4.47
Male,652,"$1,967.64",$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


# Age Demographics

In [7]:
# Establish bins for ages
no_duplicates_age = purchase_data.drop_duplicates(['SN'])

bins = [0, 9, 14, 19, 24, 29, 34, 39, 120]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Categorize the existing players using the age bins
no_duplicates_age['Age Range'] = pd.cut(no_duplicates_age['Age'], bins, labels=group_names)
age_groups_df = no_duplicates_age.groupby("Age Range")

# Calculate the numbers and percentages by age group
age_demographics = pd.DataFrame(age_groups_df["Age"].count())
age_total = no_duplicates_age["Age"].count()
age_demographics['Percentage of Players']= (age_demographics['Age'] / age_total) * 100

# Rename columns and format
age_demographics.columns = ['Total Count' , 'Percentage of Players']
age_demographics.style.format({'Percentage of Players' : '{:.2f}'})


/Users/nickbredahl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Total Count,Percentage of Players
Age Range,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


# Purchasing Analysis (Age)

In [8]:
# Calculate purchase count by age group
purchase_data['Age Range'] = pd.cut(purchase_data['Age'], bins, labels=group_names)
age_groups_df_2 = purchase_data.groupby("Age Range")
age_demographics_2 = pd.DataFrame(age_groups_df_2["Price"].count())

# Calculate average purchase price, total purchase value, and average total purchase per person and add these new columns
age_demographics_2_sum = pd.DataFrame(age_groups_df_2["Price"].sum())
age_total_2 = purchase_data["Price"].count()

age_demographics_2['Average Purchase Price'] = age_demographics_2_sum['Price'] / age_demographics_2['Price']
age_demographics_2['Total Purchase Value'] = age_demographics_2_sum['Price']

player_age_counts = no_duplicates_age['Age Range'].value_counts()
age_demographics_2['Avg Total Purchase per Person'] = age_demographics_2_sum['Price'] / player_age_counts

# Format column names and dollar amounts
age_demographics_2.columns = ['Purchase Count' , 'Average Purchase Price', 'Total Purchase Value', "Avg Total Purchase per Person"]
age_demographics_2.style.format({'Average Purchase Price' : '${:.2f}', 'Total Purchase Value' : '${:,.2f}', 'Avg Total Purchase per Person' : '${:.2f}'})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Range,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


# Top Spenders

In [9]:
# Calculate purchase count and total purchase value
purchase_count_SN = pd.DataFrame(purchase_data.groupby('SN')["Price"].count())
purchase_amount_SN = pd.DataFrame(purchase_data.groupby('SN')["Price"].sum())

# Merge dataframes for purchase count and total purchase value and name columns
top_spenders = pd.merge(purchase_count_SN, purchase_amount_SN, left_index = True, right_index = True)
top_spenders.columns = ['Purchase Count' , 'Total Purchase Value']

# Calculate average purchase price and add this new column
top_spenders['Average Purchase Price'] = top_spenders['Total Purchase Value'] / top_spenders['Purchase Count']

# Sort by total purchase value (descending), show only top 5 spenders, and format
top_spenders_final = top_spenders.sort_values('Total Purchase Value', ascending = False)
top_spenders_5 = top_spenders_final.head(5)
top_spenders_5.style.format({'Total Purchase Value' : '${:,.2f}','Average Purchase Price' : '${:.2f}'})

,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,$18.96,$3.79
Idastidru52,4,$15.45,$3.86
Chamjask73,3,$13.83,$4.61
Iral74,4,$13.62,$3.40
Iskadarya95,3,$13.10,$4.37


# Most Popular Items

In [10]:
# Calculate item price, purchase count, and total value grouped by Item ID and Item Name
item_purchase_count = purchase_data.groupby(['Item ID', 'Item Name']).count()["Price"]
item_price = purchase_data.groupby(['Item ID', 'Item Name']).mean()["Price"]
item_total_value = purchase_data.groupby(['Item ID', 'Item Name']).sum()["Price"]

# Create a dataframe with new column names
top_items = pd.DataFrame({"Purchase Count" : item_purchase_count, "Item Price" : item_price, "Total Purchase Value" : item_total_value})

# Sort by descending purchase count, limit to top 5 popular items, and format dollar values
top_items_final = top_items.sort_values('Purchase Count', ascending = False)
top_items_5 = top_items_final.head(5)
top_items_5.style.format({'Item Price' : '${:.2f}',"Total Purchase Value" : '${:.2f}'})


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


# Most Profitable Items

In [11]:
# Sort by descending total purchase value
profitable_items = top_items.sort_values('Total Purchase Value', ascending = False)

# Limit to top 5 profitable items
profitable_items_5 = profitable_items.head(5)

# Format dollar values
profitable_items_5.style.format({'Item Price' : '${:.2f}',"Total Purchase Value" : '${:.2f}'})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


# Three Observable Trends

1. Although the vast majority of Heroes of Pymoli players are male, the average purchase for females is greater.

2. Ages 20 to 24 make up the greatest total purchase value, but ages 35 to 39 spend the most per person on average. It could be that people in this older age group are playing, or maybe they are purchasing items for their children who are playing the game.  

3. The top 5 spenders are all between 20 and 25 years old, and 4 of them are male.

4. Some items have more than one Item ID.